# 2. Keywords

[수행 과정]
- Abstract에 포함되는 키워드 찾아서 변환하기 (언더바 처리)
- Keyword에 포함되는 것만 남기기

In [1]:
# Library
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import re
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [15]:
# Loading Data
words_data = pd.read_csv('./Data/Preprocess_Data/Author_Keywords.csv')
# abstract_data = pd.read_csv('./Data/Preprocess_Data/Abstract_preprocessing.csv')


In [46]:
abstract_data = abstract_data[['Year','Abstract2']]

## 1. Abstract에 포함되는 키워드 찾아서 저장하기

In [16]:
words_data = words_data[words_data['count']>=20]
# words_data.to_csv('Data/Preprocess_Data/up20_words.csv', index=False, encoding='utf-8-sig')

In [17]:
abstract_data.head()

NameError: name 'abstract_data' is not defined

In [51]:
len(words_data), len(abstract_data)

(36032, 2267163)

In [3]:
words_data.reset_index(drop=True, inplace=True)

### 완전일치한 부분이 있으면 공백에 _를 씌워서 바꿔주기
- ex) neural network -> neural_network

In [18]:
# 양쪽 공백 제거
words_data['word'] = words_data['word'].str.strip()

In [19]:
# words_data에 공백을 언더바(_)로 대체
words_data['word2'] = 0
for i in range(len(words_data)):
    if len(str(words_data['word'][i]).split(' ')) >1:
        words_data['word2'][i] = words_data['word'][i].replace(' ','_')
    else:
        words_data['word2'][i] = words_data['word'][i]

In [20]:
words_data['length'] = 0
for i in range(len(words_data)):
    words_data['length'][i] = len(str(words_data['word'][i]))

In [21]:
# 결과 확인
words_data.head()

,word,count,word2,length
0,machine learning,39529,machine_learning,16
1,deep learning,36992,deep_learning,13
2,neural network,18899,neural_network,14
3,cloud computing,17700,cloud_computing,15
4,classification,17340,classification,14


In [22]:
# length에 따라 정렬 (변환이 제대로 되지 않는 것 방지)
words_data.sort_values(by='length', ascending=False, inplace=True)
words_data.reset_index(drop=True, inplace=True)
words_data.head()

,word,count,word2,length
0,h.5.1 [information interfaces and presentation...,44,h.5.1_[information_interfaces_and_presentation...,122
1,human centered computing human computer intera...,37,human_centered_computing_human_computer_intera...,89
2,i.3.7 [computer graphics]: three dimensional g...,29,i.3.7_[computer_graphics]:_three_dimensional_g...,81
3,i.3.6 [computer graphics]: methodology and tec...,30,i.3.6_[computer_graphics]:_methodology_and_tec...,75
4,h.5.2 [information interfaces and presentation...,20,h.5.2_[information_interfaces_and_presentation...,75


In [23]:
# 중복되는 값 중 첫번째만 남기고 제거
words_data.drop_duplicates(subset='word2', keep='first', inplace=True)

In [24]:
len(words_data)

35402

In [129]:
# 디버깅
# k = 251232
abstract_data['Abstract3'] = ""
words_dict = dict(zip(words_data2['word'], words_data2['word2']))

for k in tqdm(range(10000)):
    abstract = str(abstract_data['Abstract2'][k])
    replaced_abstract = abstract

    for word, word2 in words_dict.items():
        replaced_abstract = replaced_abstract.replace(word, word2)

    abstract_data['Abstract3'][k] = replaced_abstract
    abstract_data['Abstract3'][k]

100%|█████████████████████████████████████| 10000/10000 [03:06<00:00, 53.66it/s]


In [139]:
# 방법 1
in_abstracts = []
words_subset = set(words_data['word2'])  # 단어 집합을 집합(Set)으로 변환하여 탐색 시간을 단축

# 정규식 패턴을 미리 생성하여 반복문 내에서 재사용
pattern = re.compile(r'\b({})\b'.format('|'.join(map(re.escape, words_subset))))

for k in tqdm(range(10000)):
    abstract = abs_df['Abstract3'][k]
    in_words = pattern.findall(abstract)
    in_abstracts.append([word for word in in_words if word in words_subset])

100%|█████████████████████████████████████| 10000/10000 [04:46<00:00, 34.93it/s]


In [133]:
# 방법 2 (빈도수 고려 안하는 버전)
in_abstracts2 = []
words_subset = set(words_data['word2'])  # 단어 집합을 집합(Set)으로 변환하여 탐색 시간을 단축

# 정규식 패턴을 미리 생성하여 반복문 내에서 재사용
# pattern = re.compile(r'\b({})\b'.format('|'.join(map(re.escape, words_subset))))

for k in tqdm(range(10000)):
    abstract = set(abs_df['Abstract3'][k].split())
    words_in_abstract = words_subset.intersection(abstract)
    in_abstracts.append(list(words_in_abstract))


100%|██████████████████████████████████| 10000/10000 [00:00<00:00, 31486.83it/s]


In [140]:
in_abstracts[0]

['a',
 'method',
 'peer_to_peer_streaming',
 'node',
 'is',
 'a',
 'problem',
 'is',
 'disk',
 'user',
 'content',
 'be',
 'a',
 'malicious_user',
 'content',
 'method',
 'ha',
 'advantage',
 'user',
 's',
 'disk',
 'a',
 'version',
 'content',
 'version',
 'in',
 'disk',
 'can',
 'be',
 'in',
 'it',
 'is',
 'content',
 'distribution',
 'it',
 'ieee']

In [137]:
len(in_abstracts[2])

61

In [141]:
# words 칼럼의 단어를 words2 칼럼의 단어로 대체
abstract_data['Abstract3'] = ""

words_dict = dict(zip(words_data2['word'], words_data2['word2']))

for k in tqdm(range(len(abstract_data))):
    abstract = str(abstract_data['Abstract2'][k])
    replaced_abstract = abstract

    for word, word2 in words_dict.items():
        replaced_abstract = replaced_abstract.replace(word, word2)

    abstract_data['Abstract3'][k] = replaced_abstract

100%|██████████████████████████████| 2267163/2267163 [13:21:24<00:00, 47.15it/s]


In [142]:
new_abstract = abstract_data[['Year','Abstract3']]
new_abstract.to_csv('Data/Preprocess_Data/Abstract3.csv', encoding='utf-8-sig', index=True)

In [14]:
new_abstract['Abstract3'][0]

"a method for peer_to_peer_streaming of video_on_demand_with residential node is described a possible problem with doing peer_to_peer_video_on_demand_streaming is the necessity of storing on the disk of the residential user the content to be streamed allowing a malicious_user to distribute illegally the content the proposed method ha the advantage of storing on the user's disk only a reduced version of the content although the reduced version stored in disk can still be used in the proposed peer_to_peer_scheme it is not sufficient to recover the original content preventing an unauthorized distribution of it © 2013 ieee"

## --------

In [11]:
abs_df = pd.read_csv('./Data/Preprocess_Data/Abstract3.csv')

In [12]:
abs_df

,Unnamed: 0,Year,Abstract3
0,0,2013,a method for peer_to_peer_streaming of video_o...
1,1,2013,in this paper we discus the bacterial network_...
2,2,2013,this article treat a digital_humanity work in ...
3,3,2013,this work describes preliminary step towards n...
4,4,2013,goal extraction in learning_by_demonstration i...
...,...,...,...
2267158,2267158,2021,human_centered development of information_syst...
2267159,2267159,2021,the computing device in cloud or fog data_cent...
2267160,2267160,2021,mobile_technology are becoming more and more a...
2267161,2267161,2021,development of intelligent_system with the pur...


In [26]:
in_abstracts = []
words_subset = set(words_data['word2'])  # 단어 집합을 집합(Set)으로 변환하여 탐색 시간을 단축

# 정규식 패턴을 미리 생성하여 반복문 내에서 재사용
pattern = re.compile(r'\b({})\b'.format('|'.join(map(re.escape, words_subset))))

for k in tqdm(range(1300000,1500000))
    abstract = abs_df['Abstract3'][k]
    in_words = pattern.findall(abstract)
    in_abstracts.append([word for word in in_words if word in words_subset])

100%|█████████████████████████████████| 200000/200000 [1:49:15<00:00, 30.51it/s]


In [27]:
abs_df2 = pd.DataFrame(in_abstracts)
# 각 행의 리스트를 공백으로 구분하여 연결
abs_df2 = abs_df2.apply(lambda row: ', '.join(row.dropna().astype(str)), axis=1)
# 이를 DataFrame으로 변환
abs_df2 = pd.DataFrame(abs_df2, columns=['list'])
abs_df2

,list
0,"in, paper, a, mathematical_model, is, distribu..."
1,"in, paper, a, flow_network, backhaul, path_pla..."
2,"in, dna_strand_displacement, technology, ha, a..."
3,"development, internet_of_thing, ha, growth, po..."
4,"method, dna_strand_displacement, static, think..."
...,...
199995,"in, paper, a, framework, classifying, change, ..."
199996,"present, outlier_removal, clustering_algorithm..."
199997,"in, paper, problem, a, population, mobile_robo..."
199998,"data_modeling, web_application, need, be, requ..."


In [52]:
len(abs_df2)

1000000

In [28]:
abs_df2.to_csv('Data/Preprocess_Data/추출본(1300000~1500000).csv', encoding='utf-8-sig', index=False)

In [26]:
for k in tqdm(range(500000,1000000)):
    abstract = abs_df['Abstract3'][k]
    in_words = pattern.findall(abstract)
    in_abstracts.append([word for word in in_words if word in words_subset])

100%|█████████████████████████████████| 500000/500000 [4:08:16<00:00, 33.57it/s]


In [ ]:
# for k in tqdm(range(1000000,1600000)):
#     abstract = abs_df['Abstract3'][k]
#     in_words = pattern.findall(abstract)
#     in_abstracts.append([word for word in in_words if word in words_subset])

In [ ]:
# for k in tqdm(range(1600000,len(abs_df))):
#     abstract = abs_df['Abstract3'][k]
#     in_words = pattern.findall(abstract)
#     in_abstracts.append([word for word in in_words if word in words_subset])

In [53]:
in_abstracts2 = []
for k in tqdm(range(2128315,len(abs_df))):
    abstract = abs_df['Abstract3'][k]
    in_words = pattern.findall(abstract)
    in_abstracts2.append([word for word in in_words if word in words_subset])

100%|█████████████████████████████████| 138848/138848 [1:10:10<00:00, 32.98it/s]


In [54]:
abs_df2 = pd.DataFrame(in_abstracts2)
# 각 행의 리스트를 공백으로 구분하여 연결
abs_df2 = abs_df2.apply(lambda row: ', '.join(row.dropna().astype(str)), axis=1)
# 이를 DataFrame으로 변환
abs_df2 = pd.DataFrame(abs_df2, columns=['list'])
abs_df2

,list
0,"choice, effective, management, decision, city,..."
1,"paper, aim, at, debate, open_government, impac..."
2,"vehicle_routing_problem_with_time_window, is, ..."
3,"convergence, information_and_communication_tec..."
4,"paper, control, approach, micro, satellite, au..."
...,...
138843,"human_centered, development, information_syste..."
138844,"computing, device, in, cloud, or, fog, data_ce..."
138845,"mobile_technology, a, pedagogical_tool, in, hi..."
138846,"development, intelligent_system, pursuit, dete..."


In [55]:
abs_df2.to_csv('Data/Preprocess_Data/추출본(2128315~).csv', encoding='utf-8-sig', index=False)

In [30]:
print(len(in_abstracts))

1000000


In [27]:
abs_df2 = pd.DataFrame(in_abstracts)
# 각 행의 리스트를 공백으로 구분하여 연결
abs_df2 = abs_df2.apply(lambda row: ', '.join(row.dropna().astype(str)), axis=1)
# 이를 DataFrame으로 변환
abs_df2 = pd.DataFrame(abs_df2, columns=['list'])
abs_df2

,list
0,"0, 2013, a method for peer_to_peer_streaming o..."
1,"1, 2013, in this paper we discus the bacterial..."
2,"2, 2013, this article treat a digital_humanity..."
3,"3, 2013, this work describes preliminary step ..."
4,"4, 2013, goal extraction in learning_by_demons..."
...,...
2267158,"2267158, 2021, human_centered development of i..."
2267159,"2267159, 2021, the computing device in cloud o..."
2267160,"2267160, 2021, mobile_technology are becoming ..."
2267161,"2267161, 2021, development of intelligent_syst..."


In [29]:
abs_df2.to_csv('Data/Preprocess_Data/추출본(~1000000).csv', encoding='utf-8-sig', index=False)

In [ ]:
df2 = abs_df2.copy()
# 문자열 분할, 공백 제거 및 새로운 칼럼에 저장
df2['list'] = df2['list'].apply(lambda x: x.split(',', 2)[2].strip())

In [ ]:
df2.to_csv('Data/Preprocess_Data/추출본.csv', encoding='utf-8-sig', index=False)

In [11]:
df2 = pd.read_csv('./Data/Preprocess_Data/추출본.csv')

In [13]:
df2['list'][0]

"a method for peer_to_peer_streaming of video_on_demand_with residential node is described a possible problem with doing peer_to_peer_video_on_demand_streaming is the necessity of storing on the disk of the residential user the content to be streamed allowing a malicious_user to distribute illegally the content the proposed method ha the advantage of storing on the user's disk only a reduced version of the content although the reduced version stored in disk can still be used in the proposed peer_to_peer_scheme it is not sufficient to recover the original content preventing an unauthorized distribution of it © 2013 ieee"

In [34]:
len(in_abstracts)

1000000

In [35]:
abs_df2

,list
0,"0, 2013, a method for peer_to_peer_streaming o..."
1,"1, 2013, in this paper we discus the bacterial..."
2,"2, 2013, this article treat a digital_humanity..."
3,"3, 2013, this work describes preliminary step ..."
4,"4, 2013, goal extraction in learning_by_demons..."
...,...
2267158,"2267158, 2021, human_centered development of i..."
2267159,"2267159, 2021, the computing device in cloud o..."
2267160,"2267160, 2021, mobile_technology are becoming ..."
2267161,"2267161, 2021, development of intelligent_syst..."
